# CFP 12-Team Playoff Selection

This notebook implements the official 12-team College Football Playoff selection protocol:

**Selection Rules:**
- **5 Automatic Bids:** Highest-ranked conference champions
- **7 At-Large Bids:** Next highest-ranked teams regardless of conference
- **Seeding:** Top 4 seeds (conference champions) receive first-round byes
- **First Round:** Seeds 5-8 host seeds 12-9 on campus
- **Quarterfinals & Beyond:** Neutral-site bowl games
- **No Reseeding:** Fixed bracket structure

In [1]:
# Cell 1: Setup and Imports
import sys
import os
import pandas as pd
import json
from pathlib import Path
from datetime import datetime
from IPython.display import HTML, display

# Add src to path
sys.path.insert(0, os.path.abspath('..'))

# Clear any cached imports
if 'src.playoff.bracket_plotly' in sys.modules:
    del sys.modules['src.playoff.bracket_plotly']

from src.playoff.bracket import (
    select_playoff_field,
    seed_playoff_teams,
    create_bracket_matchups,
    visualize_bracket,
    visualize_bracket_html
)
from src.playoff.bracket_plotly import create_interactive_bracket

# Create output directories
output_dir = Path('./data/output')
brackets_dir = output_dir / 'brackets'
exports_dir = output_dir / 'exports'
brackets_dir.mkdir(parents=True, exist_ok=True)
exports_dir.mkdir(parents=True, exist_ok=True)

print('✅ Imports loaded successfully')
print(f'Output directory: {output_dir}')

✅ Imports loaded successfully
Output directory: data/output


In [8]:
# Cell 2: Load Rankings
year = 2025
week = 15

# Load final composite rankings (already has conference, wins, losses, conf_tier)
rankings_dir = output_dir / 'rankings'
final_rankings = pd.read_csv(rankings_dir / f'composite_rankings_{year}_week{week}.csv')

# IMPORTANT: Load conference champions from notebook 04 (resume_analysis)
# This contains the results of simulated conference championship games
# Uses advanced tiebreaker logic (conf SOS, head-to-head, etc.)
print('📊 Loading conference champions from resume analysis...')
resume_rankings = pd.read_csv(rankings_dir / f'resume_rankings_{year}_week{week}.csv')

# Extract conference champion data from resume rankings
conf_champ_map = dict(zip(resume_rankings['team'], resume_rankings['conf_champ']))

# Add conference champion status to final_rankings
final_rankings['conf_champ'] = final_rankings['team'].map(conf_champ_map).fillna('No')

# Check if conference column exists, if not, get it from games data
if 'conference' not in final_rankings.columns:
    print('⚠️  Conference column not found in composite rankings, extracting from games data...')
    
    # Load games data to extract conference information
    cache_dir = Path(f'./data/cache/{year}')
    parquet_path = cache_dir / f'games_w{week}.parquet'
    csv_path = cache_dir / f'games_w{week}.csv'
    
    if parquet_path.exists():
        try:
            games_df = pd.read_parquet(parquet_path)
        except:
            games_df = pd.read_csv(csv_path)
    else:
        games_df = pd.read_csv(csv_path)
    
    # Extract conference info from first game for each team
    team_conference = {}
    for team in final_rankings['team']:
        team_games = games_df[(games_df['home_team'] == team) | (games_df['away_team'] == team)]
        if not team_games.empty:
            first_game = team_games.iloc[0]
            conference = first_game['home_conference'] if first_game['home_team'] == team else first_game['away_conference']
            team_conference[team] = conference
    
    final_rankings['conference'] = final_rankings['team'].map(team_conference)

# Verify we have conference data
if 'conference' not in final_rankings.columns:
    raise ValueError('Conference column missing!')

# Count conference champions
champ_count = final_rankings['conf_champ'].str.contains('Yes', na=False).sum()
print(f'   ✅ Loaded {champ_count} conference champions from championship game simulations')

print(f'\nSeason: {year} (2025-2026)')
print(f'Week: {week}')
print(f'Total teams: {len(final_rankings)}')
print(f'Top team: #{1} {final_rankings.iloc[0]["team"]}')
print()
print('Top 10 Rankings:')
print(final_rankings[['team', 'rank', 'wins', 'losses', 'composite_score']].head(10))
print()
print(f'Conference champions found: {champ_count}')

📊 Loading conference champions from resume analysis...
   ✅ Loaded 9 conference champions from championship game simulations

Season: 2025 (2025-2026)
Week: 15
Total teams: 136
Top team: #1 Ohio State

Top 10 Rankings:
         team  rank  wins  losses  composite_score
0  Ohio State     1    11       0         0.976192
1     Indiana     2    11       0         0.967141
2         BYU     3    10       1         0.892947
3      Oregon     4    10       1         0.864353
4  Texas Tech     5    10       1         0.849449
5     Georgia     6    10       1         0.844179
6   Texas A&M     7    10       1         0.823748
7    Ole Miss     8    10       1         0.805475
8    Oklahoma     9     9       2         0.797564
9  Notre Dame    10    10       2         0.795844

Conference champions found: 9


---

## Step 1: Select 12-Team Playoff Field

Apply the 5+7 protocol to select the playoff field.

In [9]:
# Cell 3: Select Playoff Field using 5+7 Protocol
# Ensure rankings are sorted
final_rankings_sorted = final_rankings.sort_values('rank').reset_index(drop=True)

# Select playoff field
selection = select_playoff_field(
    rankings_df=final_rankings_sorted,
    conference_col='conference',
    conf_champ_col='conf_champ',
    n_auto_bids=5,
    n_at_large=7
)

# Display audit log
print('=' * 80)
print('PLAYOFF SELECTION AUDIT LOG')
print('=' * 80)
for log_entry in selection.audit_log:
    print(log_entry)

# Check for displaced team
if selection.champ_pulled_in:
    print()
    print('!' * 80)
    print('SPECIAL CASE: Conference champion outside Top 12 pulled into field')
    print('!' * 80)
    if selection.displaced_team:
        print(f'Displaced team: #{selection.displaced_team["rank"]} {selection.displaced_team["team"]}')

PLAYOFF SELECTION AUDIT LOG
Found 9 conference champions

Automatic bids (top 5 conference champions):
  1. #1 Ohio State (Yes (Big Ten))
  2. #5 Texas Tech (Yes (Big 12))
  3. #6 Georgia (Yes (SEC))
  4. #16 North Texas (Yes (American Athletic))
  5. #19 James Madison (Yes (Sun Belt))

At-large bids (7 spots):
  1. #2 Indiana
  2. #3 BYU
  3. #4 Oregon
  4. #7 Texas A&M
  5. #8 Ole Miss
  6. #9 Oklahoma
  7. #10 Notre Dame

CHAMPION PULLED IN: #16 North Texas
DISPLACED: #10 Notre Dame

Final 12-team playoff field:
  1. #1 Ohio State (AUTO)
  2. #2 Indiana (AT-LARGE)
  3. #3 BYU (AT-LARGE)
  4. #4 Oregon (AT-LARGE)
  5. #5 Texas Tech (AUTO)
  6. #6 Georgia (AUTO)
  7. #7 Texas A&M (AT-LARGE)
  8. #8 Ole Miss (AT-LARGE)
  9. #9 Oklahoma (AT-LARGE)
  10. #10 Notre Dame (AT-LARGE)
  11. #16 North Texas (AUTO)
  12. #19 James Madison (AUTO)

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
SPECIAL CASE: Conference champion outside Top 12 pulled into field
!!

---

## Step 2: Seed Playoff Teams

Top 4 conference champions receive first-round byes and seeds 1-4.

In [10]:
# Cell 4: Seed the Bracket
seeded_df = seed_playoff_teams(
    playoff_teams=selection.playoff_teams,
    auto_bid_teams=selection.auto_bids
)

print('=' * 80)
print('SEEDED 12-TEAM PLAYOFF FIELD')
print('=' * 80)
print()
print(f'{'Seed':<6} {'Team':<30} {'Rank':<6} {'Record':<10} {'Bye?':<10} {'Status'}')
print('-' * 80)

auto_bid_names = {ab['team'] for ab in selection.auto_bids}

for _, team in seeded_df.iterrows():
    record = f"{int(team.get('wins', 0))}-{int(team.get('losses', 0))}"
    bye_status = 'YES' if team['is_bye'] else 'No'
    status = 'AUTO-BID' if team['team'] in auto_bid_names else 'AT-LARGE'
    print(f"{team['seed']:<6} {team['team']:<30} #{team['rank']:<5} {record:<10} {bye_status:<10} {status}")

SEEDED 12-TEAM PLAYOFF FIELD

Seed   Team                           Rank   Record     Bye?       Status
--------------------------------------------------------------------------------
1      Ohio State                     #1     11-0       YES        AUTO-BID
2      Texas Tech                     #5     10-1       YES        AUTO-BID
3      Georgia                        #6     10-1       YES        AUTO-BID
4      North Texas                    #16    10-1       YES        AUTO-BID
5      Indiana                        #2     11-0       No         AT-LARGE
6      BYU                            #3     10-1       No         AT-LARGE
7      Oregon                         #4     10-1       No         AT-LARGE
8      Texas A&M                      #7     10-1       No         AT-LARGE
9      Ole Miss                       #8     10-1       No         AT-LARGE
10     Oklahoma                       #9     9-2        No         AT-LARGE
11     Notre Dame                     #10    10-2      

---

## Step 3: Create Bracket Matchups

In [11]:
# Cell 5: Create Bracket Matchups
first_round, all_rounds = create_bracket_matchups(seeded_df)

print('=' * 80)
print('FIRST ROUND MATCHUPS (On-Campus Sites)')
print('=' * 80)
print()

for matchup in first_round:
    print(f'Game {matchup.game_num}:')
    print(f'  Seed #{matchup.seed_low}: {matchup.team_low}')
    print(f'    @')
    print(f'  Seed #{matchup.seed_high}: {matchup.team_high} (HOST)')
    print(f'  Location: {matchup.location}')
    print()

print('=' * 80)
print('QUARTERFINALS (Bowl Games, Neutral Sites)')
print('=' * 80)
print()

for matchup in all_rounds['quarterfinals']:
    print(f'QF {matchup.game_num}:')
    print(f'  Seed #{matchup.seed_high}: {matchup.team_high}')
    print(f'    vs')
    print(f'  {matchup.team_low}')
    print(f'  Location: {matchup.location}')
    print()

FIRST ROUND MATCHUPS (On-Campus Sites)

Game 1:
  Seed #12: James Madison
    @
  Seed #5: Indiana (HOST)
  Location: Campus of #5 seed

Game 2:
  Seed #11: Notre Dame
    @
  Seed #6: BYU (HOST)
  Location: Campus of #6 seed

Game 3:
  Seed #10: Oklahoma
    @
  Seed #7: Oregon (HOST)
  Location: Campus of #7 seed

Game 4:
  Seed #9: Ole Miss
    @
  Seed #8: Texas A&M (HOST)
  Location: Campus of #8 seed

QUARTERFINALS (Bowl Games, Neutral Sites)

QF 1:
  Seed #1: Ohio State
    vs
  Winner 8/9
  Location: Bowl Game (Neutral Site)

QF 2:
  Seed #2: Texas Tech
    vs
  Winner 7/10
  Location: Bowl Game (Neutral Site)

QF 3:
  Seed #3: Georgia
    vs
  Winner 6/11
  Location: Bowl Game (Neutral Site)

QF 4:
  Seed #4: North Texas
    vs
  Winner 5/12
  Location: Bowl Game (Neutral Site)



---

## Visual Playoff Bracket

Interactive HTML bracket visualization.

In [12]:
# Cell 6: Display Visual Bracket
# Merge additional ranking data for tooltips
seeded_with_stats = seeded_df.merge(
    final_rankings[['team', 'sor_rank', 'sos_rank', 'predictive_rank']],
    on='team',
    how='left'
)

# Generate interactive Plotly bracket
fig = create_interactive_bracket(seeded_with_stats, first_round)

if fig:
    # Display interactive Plotly bracket
    fig.show()
    
    # Save as HTML
    plotly_html_path = brackets_dir / f'playoff_bracket_interactive_{year}_week{week}.html'
    fig.write_html(plotly_html_path)
    print(f'✅ Interactive Plotly bracket saved: {plotly_html_path}')
else:
    print('⚠️  Plotly not available, falling back to static HTML bracket display')

# Generate HTML bracket for export (always needed)
html_bracket = visualize_bracket_html(seeded_df, first_round)

# Also show ASCII version
print()
print('=' * 80)
print('ASCII BRACKET')
print('=' * 80)
print()
ascii_bracket = visualize_bracket(seeded_df, first_round, f'{year} CFP Bracket - Week {week}')
print(ascii_bracket)

✅ Interactive Plotly bracket saved: data/output/brackets/playoff_bracket_interactive_2025_week15.html

ASCII BRACKET

                           2025 CFP Bracket - Week 15                           

FIRST ROUND BYES:
--------------------------------------------------------------------------------
  Seed #1: Ohio State                     (Rank #1)
  Seed #2: Texas Tech                     (Rank #5)
  Seed #3: Georgia                        (Rank #6)
  Seed #4: North Texas                    (Rank #16)

FIRST ROUND (On-Campus Sites):
--------------------------------------------------------------------------------
Game 1:
  Seed #5: Indiana
    vs
  Seed #12: James Madison
  Location: Campus of #5 seed

Game 2:
  Seed #6: BYU
    vs
  Seed #11: Notre Dame
  Location: Campus of #6 seed

Game 3:
  Seed #7: Oregon
    vs
  Seed #10: Oklahoma
  Location: Campus of #7 seed

Game 4:
  Seed #8: Texas A&M
    vs
  Seed #9: Ole Miss
  Location: Campus of #8 seed

QUARTERFINALS (Bowl Games, Neutr

---

## Export Results

In [13]:
# Cell 7: Export Bracket and Results
# Save HTML bracket
html_path = brackets_dir / f'playoff_bracket_{year}_week{week}.html'
with open(html_path, 'w') as f:
    f.write(f'''<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>{year} CFP Bracket - Week {week}</title>
</head>
<body>
{html_bracket}
</body>
</html>''')

# Save ASCII bracket
ascii_path = brackets_dir / f'playoff_bracket_{year}_week{week}.txt'
with open(ascii_path, 'w') as f:
    f.write(ascii_bracket)

# Save audit log
audit_path = brackets_dir / f'selection_audit_{year}_week{week}.txt'
with open(audit_path, 'w') as f:
    f.write('\n'.join(selection.audit_log))

# Prepare JSON export
playoff_json = {
    'timestamp': datetime.now().isoformat(),
    'season': year,
    'week': week,
    'selection_protocol': {
        'automatic_bids': 5,
        'at_large_bids': 7,
        'total_teams': 12,
        'bye_seeds': 4
    },
    'playoff_field': [
        {
            'seed': int(row['seed']),
            'team': row['team'],
            'rank': int(row['rank']),
            'is_bye': bool(row['is_bye']),
            'conference': row.get('conference', ''),
            'conf_champ': row.get('conf_champ', '')
        }
        for _, row in seeded_df.iterrows()
    ],
    'first_round_matchups': [
        {
            'game': m.game_num,
            'home_seed': m.seed_high,
            'home_team': m.team_high,
            'away_seed': m.seed_low,
            'away_team': m.team_low,
            'location': m.location
        }
        for m in first_round
    ],
    'automatic_bids': [
        {
            'team': ab['team'],
            'rank': int(ab['rank']),
            'conference': ab.get('conf_champ', '')
        }
        for ab in selection.auto_bids
    ],
    'at_large_bids': [
        {
            'team': al['team'],
            'rank': int(al['rank'])
        }
        for al in selection.at_large_bids
    ],
    'special_cases': {
        'champ_pulled_in': selection.champ_pulled_in,
        'displaced_team': {
            'team': selection.displaced_team['team'],
            'rank': int(selection.displaced_team['rank'])
        } if selection.displaced_team else None
    }
}

# Save JSON
json_path = brackets_dir / f'playoff_bracket_{year}_week{week}.json'
with open(json_path, 'w') as f:
    json.dump(playoff_json, f, indent=2)

# Save seeded field CSV
csv_path = exports_dir / f'playoff_field_{year}_week{week}.csv'
seeded_df.to_csv(csv_path, index=False)

print('✅ Bracket and selection data exported:')
print(f'   HTML: {html_path}')
print(f'   ASCII: {ascii_path}')
print(f'   JSON: {json_path}')
print(f'   CSV: {csv_path}')
print(f'   Audit: {audit_path}')

✅ Bracket and selection data exported:
   HTML: data/output/brackets/playoff_bracket_2025_week15.html
   ASCII: data/output/brackets/playoff_bracket_2025_week15.txt
   JSON: data/output/brackets/playoff_bracket_2025_week15.json
   CSV: data/output/exports/playoff_field_2025_week15.csv
   Audit: data/output/brackets/selection_audit_2025_week15.txt


---

## Summary

12-team College Football Playoff bracket complete!

**Outputs:**
- HTML bracket (open in browser for interactive view)
- JSON export (for external applications)
- Audit log (transparency on selection decisions)

**Next Steps:**
- `06_visualization_report.ipynb` - Stability analysis and error metrics
- `07_quick_simulator.ipynb` - Streamlined end-to-end analysis